# Chassis debugger

In [ ]:
import time
import gym
import nle

In [ ]:
del gym.Wrapper.__getattr__

We hide the NLE under several layers of wrappers. From the core to the shell:
1. `ReplayToFile` handles seeding and logs the taken actions and seed into a file for later inspection and replay.

2. `NLEObservationPatches` patches tty-screens, botched by the cr-lf misconfiguration of the NLE's tty term emulator and NetHacks displays (lf only).

3. `Chassis` handles skippable gui events that do not require a decision, such as collecting menu pages unless an interaction is required, fetching consecutive topline or log messages.

4. `ActionMasker` computes the mask of action that are **forbidden** in the current game state (_gui_ or _play_)

5. `RecentMessageLog` keeps a log of a specified number of recent messages fetched by the upstream `Chassis` wrapper.

6. `NLEAtoN` maps ascii actions to opaque actions accepted by the NLE.

In [ ]:
from nle_toolbox.bot.chassis import get_wrapper
from nle_toolbox.utils.replay import ReplayToFile, Replay
from nle_toolbox.utils.env.wrappers import NLEObservationPatches
from nle_toolbox.bot.chassis import Chassis, ActionMasker, RecentMessageLog
from nle_toolbox.utils.env.wrappers import NLEAtoN

def factory(seed=None, folder=None, sticky=False):
    env = gym.make('NetHackChallenge-v0')

    from nle.nethack import ACTIONS
    ctoa = {chr(a): j for j, a in enumerate(env.unwrapped.actions)}
    atoc = tuple(map(chr, env.unwrapped.actions))

    # provide seeding capabilities and full action tracking
    if folder is None:
        env = Replay(env, sticky=sticky)

    else:
        env = ReplayToFile(env, sticky=sticky, folder=folder, save_on='done')
    env.seed(seed)

    # patch bugged tty output
    env = NLEObservationPatches(env)

    # skippable gui abstraction layer. Bypassed if the action
    #  space does not bind a SPACE action.
    env = Chassis(env, space=ctoa.get(' '), split=False)

    # compute and action mask based on the current NLE mode: gui or play
    env = ActionMasker(env)
    return RecentMessageLog(env, n_recent=5)

A renderer for this **factory**

In [ ]:
from time import sleep

from nle_toolbox.utils.env.render import render as tty_render
from IPython.display import clear_output

def ipynb_render(obs, clear=True, fps=None):
    if fps is not None:
        if clear:
            clear_output(wait=True)

        print(tty_render(**obs))
        if fps > 0:
            sleep(fps)

    return True

We start with implementing a simple command evaluator.

In [ ]:
from collections import deque

def gui_run(env, *commands):
    pipe0 = deque([])
    obs, fin = env.reset(), False
    for cmd in commands:
        if fin:
            break

        pipe0.extend(cmd)
        while pipe0 and not fin:
            obs, rew, fin, nfo = env.step(pipe0.popleft())

        yield obs

The code below is used to debug certain events and gui

In [ ]:
import pprint as pp

def run(seed, *commands):
    with NLEAtoN(factory(seed, sticky=True)) as env:
        cha = get_wrapper(env, Chassis)
        for obs in gui_run(env, *commands):
            pp.pprint(
                (
                    cha.messages, cha.prompt,  # obs['tty_chars'][0].view('S80')[0].strip(),
                    cha.in_getlin, cha.in_menu, cha.in_yn_function, cha.xwaitingforspace,
                    obs['message_log'].view('S256')[..., 0].tolist(),
                )
            )

            ipynb_render(obs, clear=False, fps=0.01)  # dump(env.env, obs[0])

Interesting historical seeds

In [ ]:
# seed = 13765371332493407478, 12246923801353953927
# seed = 12301533412141513004, 11519511065143048485
# seed = 1632082041122464284, 11609152793318129379
# seed = 5009195464289726085, 12625175316870653325
# seed = 8962210393456991721, 8431607288866012881
# seed = 14729177660914946268, 9187177962698747861
# seed = 16892554419799916328, 6562518563582851317

# seed = 12513325507677477210, 18325590921330101247  # Ranger, arrows, dualwields
# seed = 1251332550767747710, 18325590921330101247  # Monk, martial arts, single
# seed = 125133255076774710, 18325590921330101247  # single
# seed = 14278027783296323177, 11038440290352864458  # valkyrie, dual-wield
# seed = 5009195464289726085, 12625175316870653325  # priestess, can loot lots of spells

Long multi-part message log

In [ ]:
seed = 12604736832047991440, 12469632217503715839  # Wizard, three spells, exploding wand
run(
    seed,
    '',
    ';j:',         # a paragraph about a cat
    'acy',         # break a wand "of slow" and blow up
)

Misparsed interactive menu

In [ ]:
seed = 5114808244567105441, 11072120331074183808  # Digger with a buggy backpack
run(
    seed,
    '',
    'lTb'            # pick up coins, take off leather jacket
    'ahiU $bdefg ',  # put into a sack the specified uncursed items
    'ahbb\r'         # try to take out coins
                     # <<-- FAILS, unless we add \$ to letter in `rx_menu_item`
    '$ b '
)

Interacting with a crystal orb causes an unknown prompt

In [ ]:
seed = 16441573092876245173, 16704658793745847464  # rambler who finds a crystal orb
run(
    seed,
    '',
    'bbhjJjJjj,m', # go to the crystal orb and pick it up
    'am'           # tr to peer into it
                   # <<-- FAILS with an unknown prompt
)

Teleporting into a vault and getting asked the hero's name.
* this method of getting into the vault is very finicky and RNG-dependent
on this seeded run. However, despite being seeded the game could still break
this due to its [side-effects from the real world](./nle/src/allmain.c#L53-65)
  * doesn't work during the full moon :)

In [ ]:
seed = 11245575284820074993, 10961823112882156191  # the Hatamoto ends up in a vault

into_vault = (
    'u12s\x04l'          # detect the door and kick it open
    + 'lnJnn,'           # kill the newt and pick up its corpse
    + 'nnj.h'            # reveal the trap, but it does not tele
    + 27 * '.' + 'hhh'   # wait until a zombie, then kill it
    + 35 * '.' + 'hhhl'  # wait until a newt and kill it
    + 35 * '.' + 'l'     # idle for a bit until the trap's rng
                         #  is ready to tele us into a vault
)

run(
    seed,
    '',
    into_vault,
    38 * '.' + '.',      # wait for some time until a guard shows up
    'foo\rd$h.d$.h',     # answer the prompt, drop the gold and exit
)

In [ ]:
from nle_toolbox.bot.chassis import rx_is_prompt

# some interesting messages: search for r"getlin\(" in nle's source,
#  sometimes follow `buf` or `qbuf`.
messages = [
    b"You are required to supply your name. -",
    b"What tune are you playing? [5 notes, A-G]",
    b"\"Hello stranger, who are you?\" -",
    b'What do you want to drop? [a-f or ?*]',
]
for m in messages:
    match = rx_is_prompt.search(m)
    print(match.groupdict(''))

<br>